# TensorFlow Recommenders: Matrix Factorization Model

https://www.tensorflow.org/recommenders/examples/quickstart

TFRS와 함께 [MovieLens 100K 데이터 세트](https://grouplens.org/datasets/movielens/100k/)를 사용하여 간단한 행렬 분해(matrix factorization) 모델을 구축합니다. 이 모델을 사용하여 특정 사용자에게 영화를 추천할 수 있습니다.

### Read the data

embedding layer를 위해 `사용자 ID`와 `영화 제목`을 정수 인덱스로 변환하는 vocabulary를 구축합니다.  

- `StringLookup` 레이어는 텍스트 데이터를 처리할 때 유용하게 사용됩니다. 이 경우, 사용자 ID와 영화 제목을 각각의 정수 인덱스로 매핑하여 모델이 이해할 수 있는 형태로 변환하는 역할을 합니다.  
-`adapt` 메소드는 주어진 데이터셋을 기반으로 어휘 사전을 자동으로 구성합니다. 이 과정에서 데이터셋의 모든 고유한 문자열 값을 스캔하여 인덱스를 할당합니다.  
- `mask_token=None` 옵션은 입력 데이터 중 어떤 값도 마스킹(무시)하지 않고 모두 처리하겠다는 것을 의미합니다. 일부 경우에는 특정 값을 무시하고 싶을 때 mask_token을 다르게 설정할 수 있습니다.  

이러한 어휘 사전을 사용하면 모델이 문자열 형태의 데이터를 쉽게 처리할 수 있으며, 추후에 이 데이터를 모델의 입력으로 사용할 때 일관된 방식으로 인코딩할 수 있습니다.

### model 정의

`tfrs.Model`을 상속하고 `compute_loss` 메서드를 구현하여 TFRS 모델을 정의할 수 있습니다.

두 모델과 retrieval task를 정의합니다.

tfrs.tasks.Retrieval: 이는 TFRS에서 제공하는 검색(retrieval) 작업을 설정하는 클래스입니다. 검색 작업은 사용자에게 관련 아이템을 추천하는 모델의 능력을 평가하고 최적화하는 데 중점을 둡니다.

tfrs.metrics.FactorizedTopK: 이 메트릭은 모델이 얼마나 잘 사용자에게 상위 K개의 관련 아이템을 추천하는지 평가합니다. 즉, 모델이 생성한 아이템 임베딩들 중 사용자의 실제 선호와 가장 유사한 아이템을 정확하게 찾아낼 수 있는지를 측정합니다.

movies.batch(128).map(movie_model): 이 부분은 FactorizedTopK 메트릭에 전달되는 아이템 후보 데이터셋을 준비하는 과정입니다. movies 데이터셋을 배치 크기 128로 나누고, 각 배치에 대해 movie_model을 적용하여 영화 임베딩을 생성합니다. 이렇게 생성된 임베딩들은 메트릭 계산에 사용되는 아이템 후보군으로 활용됩니다.


영화 제목에 대한 임베딩을 직접 가져올 수 있습니다


### Fit and evaluate it.

`tf.recommenders.layers.factorized_top_k`는 TensorFlow Recommenders (TFRS) 라이브러리 내에 있는 레이어 모음으로, 추천 시스템에서 사용자에게 아이템을 추천할 때 상위 K개의 가장 관련성 높은 아이템을 찾아내는 기능을 제공합니다. 이 레이어들은 추천 모델의 평가 단계에서 주로 사용되며, 모델이 얼마나 잘 사용자의 선호도를 예측하는지를 측정하는 데 도움을 줍니다.

### `factorized_top_k` 의 item 검색 방법:

1. **BruteForce**:  모든 아이템 후보에 대해 순차적으로 사용자 쿼리와의 유사성(또는 거리)을 계산하고, 가장 유사한 상위 K개의 아이템을 반환. 이 과정은 계산적으로 비효율적일 수 있지만, 작은 데이터셋에서는 실용적일 수 있습니다.

2. **ScaNN**: 대규모 데이터셋에서 더 효율적인 근사 최근접 이웃 검색을 수행하는 레이어입니다.
